In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def posConvert(paramdf):
    df = paramdf.copy()
    
    pos_dic = {'FW':[0,4], 'AM(L)':[2,3],'AM(CL)':[1,3],'AM(C)':[0,3],'AM(CR)':[1,3], 'AM(R)':[2,3],
            'M(L)':[2,2], 'M(CL)':[1,2],'M(C)':[0,2],'M(CR)':[1,2],'M(R)':[2,2], 'DMC':[0,1],
            'D(L)':[2,0], 'D(CL)':[1,0], 'D(C)':[0,0], 'D(CR)':[1,0],'D(R)':[2,0], 
           'AM(CLR)':[1,3],'M(CLR)':[1,2],'D(CLR)':[1,0],
            'AM(LR)' :[2,3],'M(LR)':[2,2], 'D(LR)':[2,0]}
    
    pos_xlist=[0*i for i in range(len(df['Position']))]
    pos_ylist=[0*i for i in range(len(df['Position']))]
    for i in range(len(df['Position'])):
        count = 0
        correction_x=False # FW는 측면 정보가 없어서 x좌표에 보정 늘거야, FW와 다른 포지션 같이 있는 데이터만
        for posdic in pos_dic.keys():
            if df['Position'][i].find(posdic) != -1:
                if posdic == 'FW':
                    correction_x = True
                pos_xlist[i] += pos_dic[posdic][0]
                pos_ylist[i] += pos_dic[posdic][1]
                count+=1
        if count == 0:
            print(df['Position'][i])

        if correction_x and count != 1:
            pos_xlist[i] += pos_xlist[i]/(count-1) # FW x값 보정, FW좌표를 뺀 나머지 포지션의 좌표 평균을 x에 더해줌
        pos_xlist[i] = pos_xlist[i]/count
        pos_ylist[i] = pos_ylist[i]/count
        
        position_df = pd.DataFrame(zip(pos_xlist,pos_ylist), columns=['posX','posY'])
    return position_df    

In [3]:
def refine(paramdf):
    df = paramdf.copy()
    df = df[df['Mins'] >= 450] # 450분 이상 출전한 자료만 
    df.drop(['OwnG'],axis=1,inplace = True)
    df.replace('-','0' , inplace=True)
    df.drop(['Apps'],axis=1,inplace=True)
    df = df[df['Position'] != 'GK'] # 골키퍼 삭제
    df = df[df['Position'] != 'Goalkeeper']
    df = df[df['Position'] != 'Substitute']
    df['Position'].replace('Forward','FW',inplace=True)
    df['Position'].replace('Midfielder','M(C)',inplace=True) # midfielder 는 M(C) 중앙 미드필더로 치환
    df['Position'].replace('Defender','D(C)',inplace=True)
    df.reset_index(inplace=True)
    df.drop(['index'],axis=1,inplace=True)
    return df

In [16]:
for i in range(11,22): # 2011-2012시즌~ 2021-2022시즌
    
    age_minus = 22-i # whoscored 에서 선수 나이가 22년도 기준으로 되어있어서 해당 년도 나이 계산하기 위해선 빼줘야댐
    
    #각 리그 스탯 및 챔스 스탯, 유에파 랭킹 가져오기
    eng = pd.read_csv('./dataset/Leagues/origin/20'+str(i)+'-20'+str(i+1)+'/England-Premier-League-20'+str(i)+'-20'+str(i+1)+'.csv', index_col=0)
    spa = pd.read_csv('./dataset/Leagues/origin/20'+str(i)+'-20'+str(i+1)+'/Spain-LaLiga-20'+str(i)+'-20'+str(i+1)+'.csv', index_col=0)
    ita = pd.read_csv('./dataset/Leagues/origin/20'+str(i)+'-20'+str(i+1)+'/Italy-Serie-A-20'+str(i)+'-20'+str(i+1)+'.csv', index_col=0)
    ger = pd.read_csv('./dataset/Leagues/origin/20'+str(i)+'-20'+str(i+1)+'/Germany-Bundesliga-20'+str(i)+'-20'+str(i+1)+'.csv', index_col=0)
    fra = pd.read_csv('./dataset/Leagues/origin/20'+str(i)+'-20'+str(i+1)+'/France-Ligue-1-20'+str(i)+'-20'+str(i+1)+'.csv', index_col=0)
    uefa = pd.read_csv('./dataset/Leagues/origin/20'+str(i)+'-20'+str(i+1)+'/Europe-Champions-League-20'+str(i)+'-20'+str(i+1)+'.csv', index_col=0)
    uefa_ranking = pd.read_csv('./dataset/UEFA/UefaRankings_5_20'+str(i+1)+'.csv',index_col=0)

    #league에 5대 리그 데이터 합치기
    uefa.reset_index(inplace=True)
    uefa.drop(['index'],axis=1,inplace=True)
    league = pd.concat([eng,spa,ita,ger,fra])
    league.reset_index(inplace=True)
    league.drop(['index'],axis=1,inplace=True)

    #league에 챔피언스리그 데이터 합치기 => league_champs라는 새로운 데이터 프레임
    league['C_Player'] = np.nan
    league['C_Mins'] = np.nan
    league['C_Tackles'] = np.nan
    league['C_Inter'] = np.nan
    league['C_Goals'] = np.nan
    league['C_Assists'] = np.nan
    league['C_KeyP'] = np.nan
    league['C_AvgP'] = np.nan
    league['C_PS%'] = np.nan

    for l_index in range(len(league)):
        for u_index in range(len(uefa)):
            if uefa.loc[u_index,'Player.1'] == league.loc[l_index,'Player.1']:
                league.loc[l_index,'C_Player'] = uefa.loc[u_index,'Player.1']
                league.loc[l_index,'C_Mins'] = uefa.loc[u_index,'Mins']
                league.loc[l_index,'C_Tackles'] = uefa.loc[u_index,'Tackles']
                league.loc[l_index,'C_Inter'] = uefa.loc[u_index,'Inter']
                league.loc[l_index,'C_Goals'] = uefa.loc[u_index,'Goals']
                league.loc[l_index,'C_Assists'] = uefa.loc[u_index,'Assists']
                league.loc[l_index,'C_KeyP'] = uefa.loc[u_index,'KeyP']
                league.loc[l_index,'C_AvgP'] = uefa.loc[u_index,'AvgP']
                league.loc[l_index,'C_PS%'] = uefa.loc[u_index,'PS%']
                league.loc[l_index,'C_Rating'] = uefa.loc[u_index,'Rating']


    #리그-챔스 스탯 = > league_champs
    league_champs = league.dropna().copy()
    league_champs.reset_index(inplace=True)
    league_champs.drop(['index'],axis=1,inplace=True)

    clublist_df=pd.read_csv('./dataset/Club_list_utf8sig.csv').dropna()
    club_series= clublist_df['Name']
    uefaclub_series=clublist_df['uefa_Name']
    
    position=[]
    age=[]
    club=[]
    name=[]
    uefa_club=[]
    
    for info in league_champs['Player.1']:
        position.append(info.split()[-1])
        age.append(int(info.split()[-2].replace(',',''))-age_minus)
        for check in club_series:
            n = info.find(' '+check+',')
            if n != -1:
                club.append(check)                
                name.append(info[:n])
    
    for clubname in club:
        uefa_club.append(clublist_df[clublist_df['Name']==clubname].iloc[0]['uefa_Name'])    

    df= pd.concat([pd.DataFrame(name,columns=['Name']),
                   pd.DataFrame(club,columns=['Club']),
                   pd.DataFrame(age,columns=['Age']),
                   pd.DataFrame(uefa_club,columns=['uefa_club']),
                   pd.DataFrame(position,columns=['Position'])],axis=1)

    df[['CurrentPts','Pts']]=np.nan
    uefa_ranking.reset_index(inplace=True)
    uefa_ranking.drop(['index'],axis=1,inplace=True)


    # uefa ranking 붙이기
    pts =[]
    currentpts=[]
    for df_index in range(len(df)):
        for uefa_index in range(len(uefa_ranking)):
            if df.loc[df_index,'uefa_club'] == uefa_ranking['Club'][uefa_index]:  
                df.loc[df_index,'CurrentPts'] = uefa_ranking.loc[uefa_index,uefa_ranking.columns[-3]]
                df.loc[df_index,'Pts'] = uefa_ranking.loc[uefa_index,uefa_ranking.columns[-2]]

    #파일 저장
    final = pd.concat([league_champs,df],axis=1)
    appended = pd.concat([refine(final),posConvert(refine(final))],axis=1)
    appended.to_csv('./dataset/appended/appendedData_20'+str(i)+'-20'+str(i+1)+'.csv',encoding='utf-8-sig')
    print('save')

save
save
save
save
save
save
save
save
save
save
save


In [12]:
clublist_df=pd.read_csv('./dataset/Club_list.csv',index_col=0).dropna()
clublist_df.to_csv('./dataset/Club_list_utf8sig.csv',encoding='utf-8-sig')

In [15]:
clublist_df=pd.read_csv('./dataset/Club_list_utf8sig.csv',index_col=0).dropna()
clublist_df

,uefa_Name
Name,
Arsenal,ARS Arsenal FC Arsenal
Everton,EVE Everton FC Everton
Man Utd,MU Manchester United Man. United
Wolves,WOL Wolverhampton Wanderers FC Wolves
West Ham,WES West Ham United FC West Ham
Liverpool,LIV Liverpool FC Liverpool
Tottenham,TOT Tottenham Hotspur Tottenham
Man City,MC Manchester City FC Man. City
Southampton,SOU Southampton FC Southampton
